# Getting Started with Promptolution

## Welcome to Promptolution! 

Discover a powerful tool for evolving and optimizing your LLM prompts. This notebook provides a friendly introduction to Promptolution's core functionality.

We're excited to have you try Promptolution - let's get started!

## Installation
Install Promptolution with a single command

In [ ]:
! pip install promptolution[api]

## Imports

In [ ]:
import pandas as pd
from promptolution import ExperimentConfig, run_experiment
import nest_asyncio
nest_asyncio.apply() # Required for notebook environments

## Setting Up Your Experiment

### Prepare the data

Below, we're using a subsample of the subjectivity dataset from Hugging Face as an example. When using your own dataset, simply ensure you name the input column "x" and the target column "y", and provide a brief description of your task, that will parsed to the meta-llm during optimization.

In [3]:
df = pd.read_csv("hf://datasets/tasksource/subjectivity/train.csv").sample(500)
df = df.rename(columns={"Sentence": "x", "Label": "y"})
df = df.replace({"OBJ": "objective", "SUBJ": "subjective"})

task_description = "The dataset contains sentences labeled as either subjective or objective. "\
        "The task is to classify each sentence as either subjective or objective. " \
        "The class mentioned first in the response of the LLM will be the prediction."

### Creating Inital Prompts

We've defined some starter prompts below, but feel free to experiment! You might also want to explore create_prompts_from_samples to automatically generate initial prompts based on your data.

In [4]:
init_prompts = [
    'Classify the given text as either an objective or subjective statement based on the tone and language used: e.g. the tone and language used should indicate whether the statement is a neutral, factual summary (objective) or an expression of opinion or emotional tone (subjective). Include the output classes "objective" or "subjective" in the prompt.',
    'What kind of statement is the following text: [Insert text here]? Is it <objective_statement> or <subjective_statement>?',
    'Identify whether a sentence is objective or subjective by analyzing the tone, language, and underlying perspective. Consider the emotion, opinion, and bias present in the sentence. Are the authors presenting objective facts or expressing a personal point of view? The output will be either "objective" (output class: objective) or "subjective" (output class: subjective).',
    'Classify the following sentences as either objective or subjective, indicating the name of the output classes: [input sentence]. Output classes: objective, subjective',
    '_query a text about legal or corporate-related issues, and predict whether the tone is objective or subjective, outputting the corresponding class "objective" for non-subjective language or "subjective" for subjective language_',
    'Classify a statement as either "subjective" or "objective" based on whether it reflects a personal opinion or a verifiable fact. The output classes to include are "objective" and "subjective".',
    'Classify the text as objective or subjective based on its tone and language.',
    'Classify the text as objective or subjective based on the presence of opinions or facts. Output classes: objective, subjective.',
    'Classify the given text as objective or subjective based on its tone, focusing on its intention, purpose, and level of personal opinion or emotional appeal, with outputs including classes such as objective or subjective.',
    "Categorize the text as either objective or subjective, considering whether it presents neutral information or expresses a personal opinion/bias.\n\nObjective: The text has a neutral tone and presents factual information about the actions of Democrats in Congress and the union's negotiations.\n\nSubjective: The text has a evaluative tone and expresses a positive/negative opinion/evaluation about the past performance of the country.",
    'Given a sentence, classify it as either "objective" or "subjective" based on its tone and language, considering the presence of third-person pronouns, neutral language, and opinions. Classify the output as "objective" if the tone is neutral and detached, focusing on facts and data, or as "subjective" if the tone is evaluative, emotive, or biased.',
    'Identify whether the given sentence is subjective or objective, then correspondingly output "objective" or "subjective" in the form of "<output class>, (e.g. "objective"), without quotes. Please note that the subjective orientation typically describes a sentence where the writer expresses their own opinion or attitude, whereas an objective sentence presents facts or information without personal involvement or bias. <output classes: subjective, objective>'
]

### Configure Your LLM

Promptolution offers three flexible ways to access language models:

1. Local LLMs (using the Transformers library)
1. vLLM backend (for efficient serving of large language models)
1. API-based LLMs (compatible with any provider following the OpenAI standard)

For this demonstration, we'll use the DeepInfra API, but you can easily switch to other providers like Anthropic or OpenAI by simply changing the base_url and llm string in the configuration.

In [ ]:
token = "YOUR_API_KEY" # Replace with your Promptolution API key

Here's an explanation of each configuration parameter in the ExperimentConfig:
- `optimizer`: The algorithm used for prompt optimization. Currently we support "capo", "evopromptga", "evopromptde", and "opro". For this example, we use "capo" as it is capable of leveraging few-shot examples.
- `task_description`: A string describing the task you're optimizing prompts for. This is used to provide the meta-llm with context about your task.
- `prompts`: A list of initial prompt strings that will be used as the starting point for optimization.
- `n_steps`: The number of optimization steps to run. Higher values allow more exploration and refinement but require more API calls and computational resources.
- `api_url`: The API endpoint URL used to access the language model. This example uses DeepInfra's API which follows the OpenAI standard.
- `llm`: The LLM to use for the experiment, as both downstream and meta LLM.
- `token`: Your API authentication token required to access the language model service.

In [ ]:
config = ExperimentConfig(
    optimizer="capo",
    task_description=task_description,
    prompts=init_prompts,
    n_steps=10,
    api_url="https://api.deepinfra.com/v1/openai",
    llm="meta-llama/Meta-Llama-3-8B-Instruct",
    token=token,
)

## Run Your Experiment

With everything configured, you're ready to optimize your prompts! The `run_experiment` function will run the optimization and evaluate on a holdout set. You can expect this cell to take a few minutes to run.

In [ ]:
prompts = run_experiment(df, config)

As you can see, most optimized prompts are semantically very similar, however they often differ heavily in performance. This is exactly what we observed in our experiments across various LLMs and datasets. Running prompt optimization is an easy way to gain significant performance improvements on your task for free!

If you run into any issues while using Promptolution, please feel free to contact us. We're also happy to receive support through pull requests and other contributions to the project.


Happy prompt optimizing! 🚀✨ We can't wait to see what you build with Promptolution! 🤖💡

In [10]:
prompts

,prompt,score
0,"Classify each sentence according to its linguistic attributes, distilling whether it embodies a detached, factual narrative or a personal, opinionated viewpoint. Pay attention to the tone, lexical choices, and underlying perspective of each sentence, teasing apart verifiable facts from emotive convictions. The predicted classification will be the leading response, with the corresponding label nestled between <final_answer> and </final_answer> markers, limited to ""objective"" and ""subjective"". Sentence analysis will identify objective presentations as neutral, straightforward conveyors of verifiable information and subjective ones as vessels for personal opinions, biases, or emotional influences.\n\nInput:",0.766667
1,"Classify each sentence in the dataset as either '<final_answer>objective</final_answer>' or '<final_answer>subjective</final_answer>' based on its linguistic tone, language used, and underlying authorial perspective. Identify whether the sentence presents factual, neutral information or expresses a personal opinion, bias, or emotional perspective. Consider the presence of emotion, opinion, and bias, as well as the tone, whether it's neutral, positive, or negative. Take into account the subtle nuances in language that distinguish objective from subjective text, including the author's voice, choice of words, and intention. Your classification should reflect the predominant perspective presented in each sentence, with the class mentioned first in your response being the prediction.\n\nInput:\nIs this a warning of what’s to come?\n\nOutput:\n<final_answer>objective</final_answer>\n\nInput:\nShe said the hope had been to “change the whole region and finally push things forward in LGBTQ+ rights in the Balkans”.\n\nOutput:\n<final_answer>objective</final_answer>\n\nInput:",0.666667
2,"Transform the given sentence into its corresponding objective or subjective classification. To achieve this, you will need to analyze the tone, purpose, and level of personal opinion or emotional appeal within each sentence. We're looking for a prediction that addresses the following inquiry: is the given sentence founded on verifiable facts, presenting information in a neutral or detached manner, or does it contain personal opinions, emotions, or biases that influence the message?\n\n Please indicate your response by specifying one of the two classes first: ""objective"" or ""subjective"". Use the following markers to guide your output: <final_answer> and </final_answer>.\n\nInput:",0.633333
3,"In the realm of corporate and legal discourse, we need to accurately categorize sentences as either objective or subjective. Objectivity is achieved when providing factual information or recounting events in a neutral tone, while subjectivity arises from evaluative or interpretative opinions. Review each sentence for its tone, purpose, and emotional appeal, analyzing whether it presents verifiable facts or conveys personal opinions and biases. Based on the Large Language Model's response, identify the primary class label and extract it from the text between '<final_answer>' and '</final_answer>'. Your prediction should be a standalone statement indicating the classified class, such as ""<final_answer>objective</final_answer>"" or ""<final_answer>subjective</final_answer>"".\n\nInput:",0.633333
4,"Categorize the text as either objective or subjective, considering whether it presents neutral information or expresses a personal opinion/bias.\n\nObjective: The text has a neutral tone and presents factual information about the actions of Democrats in Congress and the union's negotiations.\n\nSubjective: The text has a evaluative tone and expresses a positive/negative opinion/evaluation about the past performance of the country.\n\nInput:\nTecovirimat is still available for doctors to obtain for their patients through an “expanded access” protocol set up by the CDC.\n\nOutput:\n<final_answer>objective</final_answer>\n\nInput:\nFor now